<a href="https://colab.research.google.com/github/darrengao628/fine/blob/main/Fine_tune_ESM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers evaluate datasets requests pandas sklearn huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 

ERROR: unknown command "evaluate"


In [2]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
def data_prep(seq):# Data prepation 
    import pandas as pd
    df=pd.read_csv(seq)
    df.dropna()
    cyto=df["Subcellular location [CC]"].str.contains("Cytosol") | df["Subcellular location [CC]"].str.contains("Cytoplasm")
    mem=df["Subcellular location [CC]"].str.contains("Membrane") | df["Subcellular location [CC]"].str.contains("Cell membrane")
    cyto_df=df[cyto& ~mem]
    cyto_seq=cyto_df["Sequence"].tolist()
    cyto_labels=[0 for protein in cyto_seq]
    mem_df=df[mem& ~cyto]
    mem_seq=mem_df["Sequence"].tolist()
    mem_labels=[1 for protein in mem_seq]
    seq = cyto_seq + mem_seq
    labels=cyto_labels + mem_labels
    return seq,labels

In [4]:
def data_prep(seq_path):# Data prepation 
    import pandas as pd
    df=pd.read_csv(seq_path)
    seq=df["seq"].tolist()
    labels=df["labels"].tolist()
    return seq,labels

In [5]:
seq,labels=data_prep("/content/100_seq.csv")
len(labels),len(seq)

(100, 100)

In [6]:
def data_loader_prep (seq, labels,model_checkpoint):
    from sklearn.model_selection import train_test_split
    from transformers import AutoTokenizer
    from datasets import Dataset
    train_seq, test_seq, train_labels, test_labels = train_test_split(seq, labels, test_size =0.25,shuffle=True)
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    train_tokenized=tokenizer(train_seq)
    test_tokenized=tokenizer(test_seq)
    train_ds=Dataset.from_dict(train_tokenized)
    test_ds=Dataset.from_dict(test_tokenized)
    train_ds = train_ds.add_column("labels", train_labels)
    test_ds = test_ds.add_column("labels", test_labels)
    return train_ds, train_labels, test_ds,test_labels
    

In [7]:
model_checkpoint="facebook/esm2_t12_35M_UR50D"
train_ds, train_labels, test_ds,test_labels = data_loader_prep (seq, labels, model_checkpoint)
train_ds,test_ds 

(Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 75
 }), Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 25
 }))

In [8]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer
num_labels = max(train_labels + test_labels) + 1  # Add 1 since 0 can be a label
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at facebook/esm2_t12_35M_UR50D were not used when initializing EsmForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing EsmForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t12_35M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'cl

In [9]:
model_name=model_checkpoint.split('/')[-1]

In [10]:
batch_size = 8

args = TrainingArguments(
    f"{model_name}_Finetuned_localization_March_14_colab_100_seq_Epochs_50",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=50,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

from evaluate import load
import numpy as np

metric = load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [11]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/Darren628/esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab_100_seq_Epochs_50 into local empty directory.


In [12]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 75
  Num Epochs = 50
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 500
  Number of trainable parameters = 33993843


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.524941,1.000000
2,No log,0.372177,1.000000
3,No log,0.264902,1.000000
4,No log,0.180652,1.000000
5,No log,0.119473,1.000000
6,No log,0.081859,1.000000
7,No log,0.059699,1.000000
8,No log,0.046547,1.000000
9,No log,0.037355,1.000000
10,No log,0.031138,1.000000


***** Running Evaluation *****
  Num examples = 25
  Batch size = 8
Saving model checkpoint to esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab_100_seq_Epochs_50/checkpoint-10
Configuration saved in esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab_100_seq_Epochs_50/checkpoint-10/config.json
Model weights saved in esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab_100_seq_Epochs_50/checkpoint-10/pytorch_model.bin
tokenizer config file saved in esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab_100_seq_Epochs_50/checkpoint-10/tokenizer_config.json
Special tokens file saved in esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab_100_seq_Epochs_50/checkpoint-10/special_tokens_map.json
tokenizer config file saved in esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab_100_seq_Epochs_50/tokenizer_config.json
Special tokens file saved in esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab_100_seq_Epochs_50/special_tokens_map.json
***** Running Evaluati

TrainOutput(global_step=500, training_loss=0.0507873306274414, metrics={'train_runtime': 391.2187, 'train_samples_per_second': 9.585, 'train_steps_per_second': 1.278, 'total_flos': 285351907072950.0, 'train_loss': 0.0507873306274414, 'epoch': 50.0})

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline(model="Darren628/esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab_100_seq")

In [17]:
import pandas as pd

In [18]:
cytodf=pd.read_csv("/content/cyto_seq.csv")

In [19]:
memdf=pd.read_csv("/content/mem_seq.csv")

In [ ]:
cytodf

In [ ]:
memdf

In [ ]:
seq=cytodf.loc[5,"seq"]
seq

In [ ]:
pred=pipe(seq)


In [ ]:
pred=pipe(seq)
for item in pred:
  print(item['label'])
  for d in item:
    print(d)



In [ ]:
label_0_count = 0
label_1_count = 0

for i in range(200):
  seq=cytodf.loc[i,"seq"]
  print(f"Prediction the {i}st seq")
  pred=pipe(seq)
  for d in pred:
   
      if d['label'] == 'LABEL_0':
          label_0_count += 1
      elif d['label'] == 'LABEL_1':
          label_1_count += 1
print(f"LABEL_0 count: {label_0_count}")
print(f"LABEL_1 count: {label_1_count}")            
  


In [ ]:
label_0_count = 0
label_1_count = 0

for i in range(200):
  seq=memdf.loc[i,"seq"]
  pred=pipe(seq)
  for d in pred:
   
      if d['label'] == 'LABEL_0':
          label_0_count += 1
      elif d['label'] == 'LABEL_1':
          label_1_count += 1
print(f"LABEL_0 count: {label_0_count}")
print(f"LABEL_1 count: {label_1_count}")  

In [ ]:
# Test 200 seq model

# Test 200 seq model

---




In [ ]:
pip_200 = pipe = pipeline(model="Darren628/esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab")

In [ ]:
pip_full = pipe = pipeline(model="Darren628/esm2_t12_35M_UR50D_Finetuned_localization_March_9")

In [ ]:
for i in range(5):
  seq=memdf.loc[i,"seq"]
  print(f"Prediction the {i}st seq")
  pred=pip_full(seq)
  print(pred)

In [ ]:
for i in range(5):
  seq=cytodf.loc[i,"seq"]
  print(f"Prediction the {i}st seq")
  pred=pip_full(seq)
  print(pred)

# Testing 100 seq model trained with 50 epochs

In [14]:
from transformers import pipeline

In [15]:
pip_100_seq_50_epochs = pipeline(model="Darren628/esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Darren628--esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab/snapshots/23b708cb326b905b3035b01353fb3972a7074cb3/config.json
Model config EsmConfig {
  "_name_or_path": "Darren628/esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab",
  "architectures": [
    "EsmForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "classifier_dropout": null,
  "emb_layer_norm_before": false,
  "esmfold_config": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 480,
  "initializer_range": 0.02,
  "intermediate_size": 1920,
  "is_folding_model": false,
  "layer_norm_eps": 1e-05,
  "mask_token_id": 32,
  "max_position_embeddings": 1026,
  "model_type": "esm",
  "num_attention_heads": 20,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "rotary",
  "problem_type": "single_label_classification",
  "token_dropout": true,
  "torch_dtype

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--Darren628--esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab/snapshots/23b708cb326b905b3035b01353fb3972a7074cb3/pytorch_model.bin
All model checkpoint weights were used when initializing EsmForSequenceClassification.

All the weights of EsmForSequenceClassification were initialized from the model checkpoint at Darren628/esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab.
If your task is similar to the task the model of the checkpoint was trained on, you can already use EsmForSequenceClassification for predictions without further training.


loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--Darren628--esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab/snapshots/23b708cb326b905b3035b01353fb3972a7074cb3/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--Darren628--esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab/snapshots/23b708cb326b905b3035b01353fb3972a7074cb3/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--Darren628--esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab/snapshots/23b708cb326b905b3035b01353fb3972a7074cb3/tokenizer_config.json


In [20]:
for i in range(5):
  seq=memdf.loc[i,"seq"]
  print(f"Prediction the {i}st seq")
  pred=pip_100_seq_50_epochs(seq)
  print(pred)

Prediction the 0st seq
[{'label': 'LABEL_1', 'score': 0.9087384343147278}]
Prediction the 1st seq
[{'label': 'LABEL_1', 'score': 0.9070250988006592}]
Prediction the 2st seq
[{'label': 'LABEL_1', 'score': 0.9086588025093079}]
Prediction the 3st seq
[{'label': 'LABEL_1', 'score': 0.9074399471282959}]
Prediction the 4st seq
[{'label': 'LABEL_1', 'score': 0.9087242484092712}]


In [22]:
for i in range(5):
  seq=cytodf.loc[i,"seq"]
  print(f"Prediction the {i}st seq")
  pred=pip_100_seq_50_epochs(seq)
  print(pred)

Prediction the 0st seq
[{'label': 'LABEL_1', 'score': 0.5689530372619629}]
Prediction the 1st seq
[{'label': 'LABEL_0', 'score': 0.5081300735473633}]
Prediction the 2st seq
[{'label': 'LABEL_0', 'score': 0.5084296464920044}]
Prediction the 3st seq
[{'label': 'LABEL_0', 'score': 0.5066714286804199}]
Prediction the 4st seq
[{'label': 'LABEL_1', 'score': 0.6046568751335144}]


In [23]:
label_0_count = 0
label_1_count = 0

for i in range(200):
  seq=memdf.loc[i,"seq"]
  pred=pip_100_seq_50_epochs(seq)
  for d in pred:
   
      if d['label'] == 'LABEL_0':
          label_0_count += 1
      elif d['label'] == 'LABEL_1':
          label_1_count += 1
print(f"LABEL_0 count: {label_0_count}")
print(f"LABEL_1 count: {label_1_count}")  

LABEL_0 count: 0
LABEL_1 count: 200


In [24]:
label_0_count = 0
label_1_count = 0

for i in range(200):
  seq=cytodf.loc[i,"seq"]
  pred=pip_100_seq_50_epochs(seq)
  for d in pred:
   
      if d['label'] == 'LABEL_0':
          label_0_count += 1
      elif d['label'] == 'LABEL_1':
          label_1_count += 1
print(f"LABEL_0 count: {label_0_count}")
print(f"LABEL_1 count: {label_1_count}")  

LABEL_0 count: 4
LABEL_1 count: 196
